### Random Baseline for CEBaB

This is one interesting baseline. For each model architecture, we basically take a randomly initialized model and evaluate CEBaB score. This is different from the `RandomExplainer` mentioned in the paper. Here, we actually have model with random weights. 

This script simply randomly initialize different models and save to disk for evaluation.

**Note**: For random initialized model, there are two ways: (1) taking the pretrained weights which is really bad at classifying things. (2) randomly initialized model. For the `LSTM` model, it is a little tricky, but I don't think there is a much difference in this case.

In [ ]:
from libs import *
from modelings.modelings_bert import *
from modelings.modelings_roberta import *
from modelings.modelings_gpt2 import *
from modelings.modelings_lstm import *

In [ ]:
"""
The following blocks will run CEBaB benchmark in
all the combinations of the following conditions.
"""
grid = {
    "seed": [42, 66, 77, 88, 99],
    "class_num": [2, 3, 5],
    "model_arch" : ["bert-base-uncased", "roberta-base", "gpt2", "lstm"]
}

keys, values = zip(*grid.items())
permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

These are the first type where we directly import pretrained weights.

In [ ]:
for i in range(len(permutations_dicts)):
    seed=permutations_dicts[i]["seed"]
    class_num=permutations_dicts[i]["class_num"]
    model_arch=permutations_dicts[i]["model_arch"]
    h_dim = 75 if "lstm" in model_arch else 192
    if model_arch == "bert-base-uncased":
        model_path = "BERT-pretrain-results"
    elif model_arch == "roberta-base":
        model_path = "RoBERTa-pretrain-results"
    elif model_arch == "gpt2":
        model_path = "gpt2-pretrain-results"
    elif model_arch == "lstm":
        model_path = "lstm-pretrain-results"
        
    output_dir = f"../proxy_training_results/{model_path}/cebab.train.train"\
                 f".alpha.1.0.beta.0.0.gemma.0.0"\
                 f".dim.{h_dim}"\
                 f".hightype.{model_arch}.Proxy.CEBaB.sa.{class_num}-class.exclusive.mode.align"\
                 f".cls.dropout.0.1.enc.dropout.0.1.seed_{seed}"
    print("outputting to: ", output_dir)
    
    config_name = None
    tokenizer_name = None
    if model_arch == "lstm":
        config_name = "bert-base-uncased"
        tokenizer_name = "bert-base-uncased"
        
    config = AutoConfig.from_pretrained(
        config_name if config_name else model_arch,
        num_labels=class_num,
        cache_dir="../huggingface_cache/",
    )
    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_name if tokenizer_name else model_arch,
        cache_dir="../huggingface_cache/",
        use_fast=True,
    )
    
    if "lstm" not in model_arch:
        if "bert-base-uncased" in model_arch:
            model_serving_module = BertForNonlinearSequenceClassification
        elif "gpt2" in model_arch:
            model_serving_module = GPT2ForNonlinearSequenceClassification
        elif "roberta" in model_arch:
            model_serving_module = RobertaForNonlinearSequenceClassification
        model = model_serving_module.from_pretrained(
            model_arch,
            from_tf=False,
            config=config,
            cache_dir="../huggingface_cache/",
        )
        # some post-editing for customized models.
        if model_arch == "gpt2":
            # Define a padding token
            model.config.pad_token_id = tokenizer.pad_token_id
    elif "lstm" in model_arch:
        config.update_embeddings=False
        config.bidirectional=True
        config.num_hidden_layers=1
        config.hidden_size=300
        model = LSTMForNonLinearSequenceClassification(
            config=config,
        )
        # load the preloaded embedding file.
        fasttext_embeddings = torch.load("../eval_pipeline/customized_models/lstm/embeddings.bin")
        model.lstm.embeddings.word_embeddings.weight.data = fasttext_embeddings
    model.save_pretrained(
        output_dir,
    )

These are the second case where all weights are randomly initialized! They are not pretrained anymore.

In [ ]:
for i in range(len(permutations_dicts)):
    seed=permutations_dicts[i]["seed"]
    class_num=permutations_dicts[i]["class_num"]
    model_arch=permutations_dicts[i]["model_arch"]
    h_dim = 75 if "lstm" in model_arch else 192
    if model_arch == "bert-base-uncased":
        model_path = "BERT-random-results"
    elif model_arch == "roberta-base":
        model_path = "RoBERTa-random-results"
    elif model_arch == "gpt2":
        model_path = "gpt2-random-results"
    elif model_arch == "lstm":
        model_path = "lstm-random-results"
        
    output_dir = f"../proxy_training_results/{model_path}/cebab.train.train"\
                 f".alpha.1.0.beta.0.0.gemma.0.0"\
                 f".dim.{h_dim}"\
                 f".hightype.{model_arch}.Proxy.CEBaB.sa.{class_num}-class.exclusive.mode.align"\
                 f".cls.dropout.0.1.enc.dropout.0.1.seed_{seed}"
    print("outputting to: ", output_dir)
    
    config_name = None
    tokenizer_name = None
    if model_arch == "lstm":
        config_name = "bert-base-uncased"
        tokenizer_name = "bert-base-uncased"
        
    config = AutoConfig.from_pretrained(
        config_name if config_name else model_arch,
        num_labels=class_num,
        cache_dir="../huggingface_cache/",
    )
    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_name if tokenizer_name else model_arch,
        cache_dir="../huggingface_cache/",
        use_fast=True,
    )
    
    if "lstm" not in model_arch:
        if "bert-base-uncased" in model_arch:
            model_serving_module = BertForNonlinearSequenceClassification
        elif "gpt2" in model_arch:
            model_serving_module = GPT2ForNonlinearSequenceClassification
        elif "roberta" in model_arch:
            model_serving_module = RobertaForNonlinearSequenceClassification
        model = model_serving_module(
            config=config,
        )
        # some post-editing for customized models.
        if model_arch == "gpt2":
            # Define a padding token
            model.config.pad_token_id = tokenizer.pad_token_id
    elif "lstm" in model_arch:
        config.update_embeddings=False
        config.bidirectional=True
        config.num_hidden_layers=1
        config.hidden_size=300
        model = LSTMForNonLinearSequenceClassification(
            config=config,
        )
        # load the preloaded embedding file.
        fasttext_embeddings = torch.load("../eval_pipeline/customized_models/lstm/embeddings.bin")
        model.lstm.embeddings.word_embeddings.weight.data = nn.Embedding(
            fasttext_embeddings.shape[0], fasttext_embeddings.shape[1]
        ).weight.data
    model.save_pretrained(
        output_dir,
    )